In [ ]:
#Project 1： Face Classfication 人脸分类
#Organize Your Dataset
data/
    train/
        cats/
            cat001.jpg
            cat002.jpg
            ...
        dogs/
            dog001.jpg
            dog002.jpg
            ...
    val/
        cats/
            cat101.jpg
            cat102.jpg
            ...
        dogs/
            dog101.jpg
            dog102.jpg
            ...

In [ ]:
#Project 1： Face Classfication 人脸分类
#use pretrained AlexNet
from torchvision import datasets, transforms
import torch
from torchvision import models
import torch.nn as nn
import torch.optim as optim

# Define transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # CIFAR-10 normalization
])

# Load the training and validation datasets
train_dataset = datasets.ImageFolder('./data/train', transform=transform)
val_dataset = datasets.ImageFolder('./data/val', transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

#Define the AlexNet Model
alexnet = models.alexnet(pretrained=True)

# Modify the classifier for binary classification
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, 2)

# If you're using GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

#Define the Loss Function and Optimizer
#criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0]))  # Adjust weights as necessary
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)
#Train the Network
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')
        evaluate_model(model, val_loader)
#Test the Model
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

train_model(alexnet, criterion, optimizer, num_epochs=20)

In [ ]:
##Project 1： Face Classfication 人脸分类 predict
from PIL import Image
import torchvision.transforms as transforms

def load_and_preprocess_image(image_path):
    # Define the same transformations as your training images
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Load the image
    image = Image.open(image_path).convert('RGB')
    # Apply the transformations
    image_tensor = transform(image)
    # Add a batch dimension (Batch Size, Color Channels, Height, Width)
    image_tensor = image_tensor.unsqueeze(0)
    return image_tensor

def evaluate_model(model, image_tensor, device):
    # Ensure model is in evaluation mode
    model.eval()
    # Send the image tensor to the same device that model is on (GPU or CPU)
    image_tensor = image_tensor.to(device)
    # Perform the forward pass and get the output predictions
    with torch.no_grad():
        outputs = model(image_tensor)
    # Convert output probabilities to predicted class
    _, predicted = torch.max(outputs, 1)
    # Retrieve the index of the highest score to get the predicted class
    predicted_class_index = predicted.item()
    return predicted_class_index

# Assuming 'alexnet' is your model and it's already loaded and set to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet.to(device)

# Load and preprocess the image
#image_path = './data/F003.jpg'  # Change this to the path of your image
image_path = './data/M006.jpg'
#image_path = './data/test.png'
image_tensor = load_and_preprocess_image(image_path)

# Evaluate the model with the image
predicted_class_index = evaluate_model(alexnet, image_tensor, device)

# Assuming you have a list or dict mapping indices to class names
classes = ['Female', 'Male', ...]  # This should be defined based on your specific classes
print("Predicted Class:", classes[predicted_class_index])